### ETL Project

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from config import username, password, dbhost, dbport, dbname 

### Extract CSVs into DataFrames

In [2]:
movies_path = os.path.join('Resources', 'movies.csv')
movies_df = pd.read_csv(movies_path, low_memory=False)
movies_df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,12/26/1906,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,"$2,250",NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,8/19/1911,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,11/13/1912,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,"$45,000",NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,3/6/1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [3]:
movies_df.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics'],
      dtype='object')

In [4]:
movies_clean_df = movies_df[['imdb_title_id', 'title', 'year', 'country', 'language']].copy()
movies_clean_df
                         

,imdb_title_id,title,year,country,language
0,tt0000009,Miss Jerry,1894,USA,None
1,tt0000574,The Story of the Kelly Gang,1906,Australia,None
2,tt0001892,Den sorte drøm,1911,"Germany, Denmark",NaN
3,tt0002101,Cleopatra,1912,USA,English
4,tt0002130,L'Inferno,1911,Italy,Italian
...,...,...,...,...,...
85850,tt9908390,Le lion,2020,"France, Belgium",French
85851,tt9911196,De Beentjes van Sint-Hildegard,2020,Netherlands,"German, Dutch"
85852,tt9911774,Padmavyuhathile Abhimanyu,2019,India,Malayalam
85853,tt9914286,Sokagin Çocuklari,2019,Turkey,Turkish


In [5]:
type(movies_clean_df)

pandas.core.frame.DataFrame

In [6]:
movies_clean_df.dtypes

imdb_title_id    object
title            object
year             object
country          object
language         object
dtype: object

In [7]:
movies_clean_df['year'] = movies_clean_df['year'].astype(str)

In [8]:
movies_clean_df = movies_clean_df[movies_clean_df['year'].str.split().str.len().lt(4)]
movies_clean_df

,imdb_title_id,title,year,country,language
0,tt0000009,Miss Jerry,1894,USA,None
1,tt0000574,The Story of the Kelly Gang,1906,Australia,None
2,tt0001892,Den sorte drøm,1911,"Germany, Denmark",NaN
3,tt0002101,Cleopatra,1912,USA,English
4,tt0002130,L'Inferno,1911,Italy,Italian
...,...,...,...,...,...
85850,tt9908390,Le lion,2020,"France, Belgium",French
85851,tt9911196,De Beentjes van Sint-Hildegard,2020,Netherlands,"German, Dutch"
85852,tt9911774,Padmavyuhathile Abhimanyu,2019,India,Malayalam
85853,tt9914286,Sokagin Çocuklari,2019,Turkey,Turkish


In [9]:
movies_clean_df.replace({'year':'TV Movie 2019'},{'year':'2019'},inplace=True)

In [10]:
movies_clean_df.loc[movies_clean_df['year']=='TV Movie 2019']

,imdb_title_id,title,year,country,language


In [11]:
movies_clean_df['year'] = movies_clean_df['year'].astype(int)

In [12]:
movies_clean_year_df = movies_clean_df[movies_clean_df['year']>2000]
movies_clean_year_df

,imdb_title_id,title,year,country,language
4334,tt0035423,Kate & Leopold,2001,USA,"English, French"
15675,tt0069049,L'altra faccia del vento,2018,"France, Iran, USA","English, German"
21806,tt0088751,The Naked Monster,2005,USA,English
24217,tt0096056,Delitto e castigo,2002,"Poland, Russia, USA","English, Polish"
25479,tt0100275,La Telenovela Errante,2017,Chile,Spanish
...,...,...,...,...,...
85850,tt9908390,Le lion,2020,"France, Belgium",French
85851,tt9911196,De Beentjes van Sint-Hildegard,2020,Netherlands,"German, Dutch"
85852,tt9911774,Padmavyuhathile Abhimanyu,2019,India,Malayalam
85853,tt9914286,Sokagin Çocuklari,2019,Turkey,Turkish


In [13]:
ratings_path = os.path.join('Resources', 'ratings.csv')
ratings_df = pd.read_csv(ratings_path)
ratings_df

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,...,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,...,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,...,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,5.3,398,5.5,6.0,13,9,26,65,104,...,5.7,11.0,5.0,2.0,5.5,12.0,6.3,22.0,5.3,214.0
85851,tt9911196,7.7,724,7.9,8.0,65,139,288,170,42,...,8.0,47.0,7.3,30.0,7.0,6.0,6.8,13.0,7.7,388.0
85852,tt9911774,7.9,265,7.8,8.0,63,29,61,61,31,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0,2.0
85853,tt9914286,6.4,194,9.4,10.0,176,0,2,2,1,...,NaN,NaN,7.0,1.0,4.0,3.0,1.7,5.0,5.8,5.0


In [14]:
ratings_df.columns

Index(['imdb_title_id', 'weighted_average_vote', 'total_votes', 'mean_vote',
       'median_vote', 'votes_10', 'votes_9', 'votes_8', 'votes_7', 'votes_6',
       'votes_5', 'votes_4', 'votes_3', 'votes_2', 'votes_1',
       'allgenders_0age_avg_vote', 'allgenders_0age_votes',
       'allgenders_18age_avg_vote', 'allgenders_18age_votes',
       'allgenders_30age_avg_vote', 'allgenders_30age_votes',
       'allgenders_45age_avg_vote', 'allgenders_45age_votes',
       'males_allages_avg_vote', 'males_allages_votes', 'males_0age_avg_vote',
       'males_0age_votes', 'males_18age_avg_vote', 'males_18age_votes',
       'males_30age_avg_vote', 'males_30age_votes', 'males_45age_avg_vote',
       'males_45age_votes', 'females_allages_avg_vote',
       'females_allages_votes', 'females_0age_avg_vote', 'females_0age_votes',
       'females_18age_avg_vote', 'females_18age_votes',
       'females_30age_avg_vote', 'females_30age_votes',
       'females_45age_avg_vote', 'females_45age_votes',
       

In [15]:
ratings_clean_df = ratings_df[['imdb_title_id', 'weighted_average_vote', 'total_votes', 'mean_vote']].copy()
ratings_clean_df

,imdb_title_id,weighted_average_vote,total_votes,mean_vote
0,tt0000009,5.9,154,5.9
1,tt0000574,6.1,589,6.3
2,tt0001892,5.8,188,6.0
3,tt0002101,5.2,446,5.3
4,tt0002130,7.0,2237,6.9
...,...,...,...,...
85850,tt9908390,5.3,398,5.5
85851,tt9911196,7.7,724,7.9
85852,tt9911774,7.9,265,7.8
85853,tt9914286,6.4,194,9.4


In [16]:
ratings_clean_df.dropna()

,imdb_title_id,weighted_average_vote,total_votes,mean_vote
0,tt0000009,5.9,154,5.9
1,tt0000574,6.1,589,6.3
2,tt0001892,5.8,188,6.0
3,tt0002101,5.2,446,5.3
4,tt0002130,7.0,2237,6.9
...,...,...,...,...
85850,tt9908390,5.3,398,5.5
85851,tt9911196,7.7,724,7.9
85852,tt9911774,7.9,265,7.8
85853,tt9914286,6.4,194,9.4


In [17]:
ratings_cleanfinal_df = ratings_clean_df.loc[(ratings_df['weighted_average_vote']>=8)]
ratings_cleanfinal_df

,imdb_title_id,weighted_average_vote,total_votes,mean_vote
165,tt0010323,8.1,55601,7.9
251,tt0012349,8.3,109038,8.2
252,tt0012364,8.1,10054,7.9
334,tt0014429,8.1,17795,8.0
372,tt0015064,8.1,12157,7.9
...,...,...,...,...
85728,tt9766160,8.4,1076,9.0
85769,tt9811598,8.3,121,9.0
85828,tt9886872,8.1,990,8.8
85843,tt9900782,8.5,8400,8.7


### Create database connection

In [18]:
connection_string = f'postgresql://{username}:{password}@{dbhost}:{dbport}/{dbname}'

engine = create_engine(connection_string)
conn = engine.connect()

### Load DataFrames into database

In [21]:
movies_clean_year_df.to_sql(name='movies', con=conn, if_exists='replace')

In [22]:
ratings_cleanfinal_df.to_sql(name='ratings', con=conn, if_exists='replace')

### Confirm Load to SQL by bringing the tables back to Pandas

In [23]:
movies_list = pd.read_sql('select * from movies', con=engine)
movies_list

,index,imdb_title_id,title,year,country,language
0,4334,tt0035423,Kate & Leopold,2001,USA,"English, French"
1,15675,tt0069049,L'altra faccia del vento,2018,"France, Iran, USA","English, German"
2,21806,tt0088751,The Naked Monster,2005,USA,English
3,24217,tt0096056,Delitto e castigo,2002,"Poland, Russia, USA","English, Polish"
4,25479,tt0100275,La Telenovela Errante,2017,Chile,Spanish
...,...,...,...,...,...,...
45788,85850,tt9908390,Le lion,2020,"France, Belgium",French
45789,85851,tt9911196,De Beentjes van Sint-Hildegard,2020,Netherlands,"German, Dutch"
45790,85852,tt9911774,Padmavyuhathile Abhimanyu,2019,India,Malayalam
45791,85853,tt9914286,Sokagin Çocuklari,2019,Turkey,Turkish


In [24]:
ratings_list = pd.read_sql('select * from ratings', con=engine)
ratings_list

,index,imdb_title_id,weighted_average_vote,total_votes,mean_vote
0,165,tt0010323,8.1,55601,7.9
1,251,tt0012349,8.3,109038,8.2
2,252,tt0012364,8.1,10054,7.9
3,334,tt0014429,8.1,17795,8.0
4,372,tt0015064,8.1,12157,7.9
...,...,...,...,...,...
1736,85728,tt9766160,8.4,1076,9.0
1737,85769,tt9811598,8.3,121,9.0
1738,85828,tt9886872,8.1,990,8.8
1739,85843,tt9900782,8.5,8400,8.7
